In [ ]:
import numpy as np
import cv2
import time
import datetime
from collections import deque
import cv2
from time import sleep
from PIL import Image 

cap = cv2.VideoCapture(0)
foog = cv2.createBackgroundSubtractorMOG2( detectShadows = True, varThreshold = 100, history = 2000)

def is_person_present(frame, thresh=1100):
    kernel = None
    
    global foog
    
    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.dilate(fgmask,kernel,iterations = 4)

    # Detect contours in the frame
    contours, hierarchy = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
     
    # Check if there was a contour and the area is somewhat higher than some threshold so we know its a person and not noise
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > thresh:
            
            # Get the max contour
            cnt = max(contours, key = cv2.contourArea)

            # Draw a bounding box around the person and label it as person detected
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
            #frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame,'Person Detected',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
            
            return True, frame
        
        
    # Otherwise report there was no one present
    else:
        return False, frame

def main_app(name="charanG",video_link=0):
        
        face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read(f"./data/classifiers/{name}_classifier.xml")
#         cap1 = cv2.VideoCapture(video_link)
        pred = 0
        status = False
        cap = cv2.VideoCapture(0)
        while True:
            
#             cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

            # This is a test video
#             cap = cv2.VideoCapture(0)

            # Read the video steram from the camera
            #cap = cv2.VideoCapture('http://192.168.18.4:8080/video')

            # Get width and height of the frame
            width = int(cap.get(3))
            height = int(cap.get(4))

            # Read and store the credentials information in a dict


            # Initialize the background Subtractor
           

            # Status is True when person is present and False when the person is not present.
            

            # After the person disapears from view, wait atleast 7 seconds before making the status False
            patience = 7

            # We don't consider an initial detection unless its detected 15 times, this gets rid of false positives
            detection_thresh = 1

            # Initial time for calculating if patience time is up
            initial_time = None

            # We are creating a deque object of length detection_thresh and will store individual detection statuses here
            de = deque([False] * detection_thresh, maxlen=detection_thresh)

            # Initialize these variables for calculating FPS
            fps = 0 
            frame_counter = 0
            start_time = time.time()

            ret, frame = cap.read()
            if not ret:
                break 

            # This function will return a boolean variable telling if someone was present or not, it will also draw boxes if it 
            # finds someone
            
            detected, annotated_image = is_person_present(frame)  

        # Register the current detection status on our deque object
            de.appendleft(detected)
            print('de1',de)


            # If we have consectutively detected a person 15 times then we are sure that soemone is present    
            # We also make this is the first time that this person has been detected so we only initialize the videowriter once
            if sum(de) == detection_thresh and not status:
                    print('de2',de)
                    status = True
                    entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
                    out = cv2.VideoWriter('outputs/{}.mp4'.format(entry_time), cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))
                    #main_app(video_link="C:\Users\Charan\Desktop\tarp_project\outputs\Thursday, 11-52-31 AM 04 August 2022")
                    #main_app()

            # If status is True but the person is not in the current frame
            
            print('detected',detected)
            if status and not detected:

                # Restart the patience timer only if the person has not been detected for a few frames so we are sure it was'nt a 
                # False positive
                if sum(de) ==0:

                    if initial_time is None:
                        initial_time = time.time()
                    status = False
                    exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                    out.release()
                    initial_time = None

#                 elif initial_time is not None:        

#                     # If the patience has run out and the person is still not detected then set the status to False
#                     # Also save the video by releasing the video writer and send a text message.
#                     if  time.time() - initial_time >= patience:
#                         status = False
#                         exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
#                         out.release()
#                         initial_time = None


            # If significant amount of detections (more than half of detection_thresh) has occured then we reset the Initial Time.
            elif status and sum(de) > (detection_thresh/2):
                initial_time = None

            # Get the current time in the required format
            current_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")

            # Display the FPS
            cv2.putText(annotated_image, 'FPS: {:.2f}'.format(fps), (510, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155),2)

            # Display Time
            cv2.putText(annotated_image, current_time, (310, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255),1)    

            # Display the Room Status
            cv2.putText(annotated_image, 'Room Occupied: {}'.format(str(status)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 10, 150),2)

            # Show the patience Value
            if initial_time is None:
                text = 'Patience: {}'.format(patience)
            else: 
                text = 'Patience: {:.2f}'.format(max(0, patience - (time.time() - initial_time)))

            cv2.putText(annotated_image, text, (10, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155) , 2)   

            # If status is true save the frame
            if status:
                out.write(annotated_image)

            # Show the Frame
#             cv2.imshow('frame',frame)

            # Calculate the Average FPS
            frame_counter += 1
            fps = (frame_counter / (time.time() - start_time))


            # Exit if q is pressed.
#             if cv2.waitKey(30) == ord('q'):
#                 break

        # Release Capture and destroy windows
#         cap.release()
#         cv2.destroyAllWindows()



            
            #-------------------------------------------------------------------------
#             ret, frame = cap.read()
            #default_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             detect_intrusion()
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray,1.3,5)

            for (x,y,w,h) in faces:


                roi_gray = gray[y:y+h,x:x+w]

                id,confidence = recognizer.predict(roi_gray)
                confidence = 100 - int(confidence)
                pred = 0
                
                if confidence > 50:
                    #if u want to print confidence level
                            #confidence = 100 - int(confidence)
                            pred += +1
                            text = name.upper()
                            font = cv2.FONT_HERSHEY_PLAIN
                            #frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            frame =  cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
                            frame = cv2.putText(frame, text, (x, y-4), font, 1, (0, 255, 0), 1, cv2.LINE_AA)

                else:   
                            pred += -1
                            text = "UnknownFace"
                            font = cv2.FONT_HERSHEY_PLAIN
                            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            frame =  cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
                            #frame = cv2.putText(frame, text, (x, y-4), font, 1, (0, 255, 0), 1, cv2.LINE_AA)

            cv2.imshow("image", frame)


            if cv2.waitKey(20) & 0xFF == ord('q'):
                print(pred)
                if pred > 0 : 
                    dim =(124,124)
                    img = cv2.imread(f".\\data\\{name}\\{pred}{name}.jpg", cv2.IMREAD_UNCHANGED)
                    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
                    cv2.imwrite(f".\\data\\{name}\\50{name}.jpg", resized)
                    Image1 = Image.open(f".\\2.png") 
                      
                    # make a copy the image so that the  
                    # original image does not get affected 
                    Image1copy = Image1.copy() 
                    Image2 = Image.open(f".\\data\\{name}\\50{name}.jpg") 
                    Image2copy = Image2.copy() 
                      
                    # paste image giving dimensions 
                    Image1copy.paste(Image2copy, (195, 114)) 
                      
                    # save the image  
                    Image1copy.save("end.png") 
                    frame = cv2.imread("end.png", 1)

                    cv2.imshow("Result",frame)
                    cv2.waitKey(5000)
                break
        
        #detect_intrusion()
        cap.release()
        #cv2.destroyAllWindows()



main_app()

#app_starts here-------------------------------------------------------------------
# Set Window normal so we can resize it


de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([False], maxlen=1)
detected False
de1 deque([True], maxlen=1)
de2 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([False], maxlen=1)
detected False
de1 deque([True], maxlen=1)
de2 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

...

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detected True
de1 deque([True], maxlen=1)
detect

In [1]:
import numpy as np
import cv2
import time
import datetime
from collections import deque
import cv2
from time import sleep
from PIL import Image 

def is_person_present(frame, thresh=1100):
    kernel = None
    
    global foog
    
    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.dilate(fgmask,kernel,iterations = 4)

    # Detect contours in the frame
    contours, hierarchy = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
     
    # Check if there was a contour and the area is somewhat higher than some threshold so we know its a person and not noise
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > thresh:
            
            # Get the max contour
            cnt = max(contours, key = cv2.contourArea)

            # Draw a bounding box around the person and label it as person detected
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
            #frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame,'Person Detected',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
            
            return True, frame
        
        
    # Otherwise report there was no one present
    else:
        return False, frame
    
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# This is a test video
cap = cv2.VideoCapture(0)

# Read the video steram from the camera
#cap = cv2.VideoCapture('http://192.168.18.4:8080/video')

# Get width and height of the frame
width = int(cap.get(3))
height = int(cap.get(4))

# Read and store the credentials information in a dict


# Initialize the background Subtractor
foog = cv2.createBackgroundSubtractorMOG2( detectShadows = True, varThreshold = 100, history = 2000)

# Status is True when person is present and False when the person is not present.
status = False

# After the person disapears from view, wait atleast 7 seconds before making the status False
patience = 7

# We don't consider an initial detection unless its detected 15 times, this gets rid of false positives
detection_thresh = 15

# Initial time for calculating if patience time is up
initial_time = None

# We are creating a deque object of length detection_thresh and will store individual detection statuses here
de = deque([False] * detection_thresh, maxlen=detection_thresh)

# Initialize these variables for calculating FPS
fps = 0 
frame_counter = 0
start_time = time.time()


while(True):
    
    ret, frame = cap.read()
    if not ret:
        break 
            
    # This function will return a boolean variable telling if someone was present or not, it will also draw boxes if it 
    # finds someone
    detected, annotated_image = is_person_present(frame)  
    
    # Register the current detection status on our deque object
    de.appendleft(detected)
    
     
    # If we have consectutively detected a person 15 times then we are sure that soemone is present    
    # We also make this is the first time that this person has been detected so we only initialize the videowriter once
    if sum(de) == detection_thresh and not status:   
            print('de',de)
            status = True
            entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
            out = cv2.VideoWriter('outputs/{}.mp4'.format(entry_time), cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))
            #main_app(video_link="C:\Users\Charan\Desktop\tarp_project\outputs\Thursday, 11-52-31 AM 04 August 2022")
            #main_app()

    # If status is True but the person is not in the current frame
    if status and not detected:
        
        # Restart the patience timer only if the person has not been detected for a few frames so we are sure it was'nt a 
        # False positive
        if sum(de) > (detection_thresh/2): 
            
            if initial_time is None:
                initial_time = time.time()
            
        elif initial_time is not None:        
            
            # If the patience has run out and the person is still not detected then set the status to False
            # Also save the video by releasing the video writer and send a text message.
            if  time.time() - initial_time >= patience:
                status = False
                exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                out.release()
                initial_time = None
            
               
    # If significant amount of detections (more than half of detection_thresh) has occured then we reset the Initial Time.
    elif status and sum(de) > (detection_thresh/2):
        initial_time = None
    
    # Get the current time in the required format
    current_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")

    # Display the FPS
    cv2.putText(annotated_image, 'FPS: {:.2f}'.format(fps), (510, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155),2)
    
    # Display Time
    cv2.putText(annotated_image, current_time, (310, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255),1)    
    
    # Display the Room Status
    cv2.putText(annotated_image, 'Room Occupied: {}'.format(str(status)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 10, 150),2)

    # Show the patience Value
    if initial_time is None:
        text = 'Patience: {}'.format(patience)
    else: 
        text = 'Patience: {:.2f}'.format(max(0, patience - (time.time() - initial_time)))
        
    cv2.putText(annotated_image, text, (10, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155) , 2)   

    # If status is true save the frame
    if status:
        out.write(annotated_image)
 
    # Show the Frame
    cv2.imshow('frame',frame)
    
    # Calculate the Average FPS
    frame_counter += 1
    fps = (frame_counter / (time.time() - start_time))
    
    
    # Exit if q is pressed.
    if cv2.waitKey(30) == ord('q'):
        break

# Release Capture and destroy windows
cap.release()
cv2.destroyAllWindows()

de deque([True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], maxlen=15)


In [ ]:
main_app(video_link='http://192.168.1.14:8080/video')

frame [[[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 [[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 [[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 136  99]
  [102 141  89]
  [102 141  89]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]]
frame [[[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 1

frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 111 111]
  [ 99 111 111]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 111 111]
  [ 99 111 111]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 111 111]
  [ 99 111 111]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 136  99]
  [102 141  89]
  [102 141  89]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]]
frame [[[148 148 148]
  [148 148 148]
  [148 148 148]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[148 148 148]
  [148 148 148]
  [148 148 148]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[147 147 147]
  [147 147 147]
  [147 147 147]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 1

frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [104 110 117]
  [104 110 116]
  [104 110 116]]

 ...

 [[130 153 150]
  [130 153 150]
  [130 153 150]
  ...
  [102 136  99]
  [102 141  89]
  [102 141  89]]

 [[130 153 150]
  [130 153 150]
  [130 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]

 [[130 153 150]
  [130 153 150]
  [130 153 150]
  ...
  [102 141  89]
  [102 147  78]
  [102 147  78]]]
frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 97 107 113]
  [ 97 107 112]
  [ 97 107 112]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 97 107 113]
  [ 97 107 112]
  [ 97 107 112]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 97 107 113]
  [ 97 107 112]
  [ 97 107 112]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 1

  [100 145  76]]]
frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [100 106 113]
  [100 106 112]
  [100 106 112]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [100 106 113]
  [100 106 112]
  [100 106 112]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [100 106 113]
  [100 106 112]
  [100 106 112]]

 ...

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 139  93]
  [102 144  82]
  [102 144  82]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 144  82]
  [102 150  71]
  [102 150  71]]

 [[134 153 150]
  [134 153 150]
  [134 153 150]
  ...
  [102 144  82]
  [102 150  71]
  [102 150  71]]]
frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 ...

 [[134 153 150]
  [134

frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 ...

 [[136 155 152]
  [136 155 152]
  [136 155 152]
  ...
  [102 138  95]
  [102 144  82]
  [102 144  82]]

 [[136 155 152]
  [136 155 152]
  [136 155 152]
  ...
  [102 144  82]
  [102 156  60]
  [102 156  60]]

 [[136 155 152]
  [136 155 152]
  [136 155 152]
  ...
  [102 144  82]
  [102 156  60]
  [102 156  60]]]
frame [[[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 99 109 115]
  [ 99 109 114]
  [ 99 109 114]]

 ...

 [[136 155 152]
  [136 155 152]
  [136 1

...

frame [[[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]]
frame [[[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 19 22]
  [32 19 22]
  [32 19 22]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 19 22]
  [32 19 22]
  [32 19 22]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 19 22]
  [32 19 22]
  [32 19 22]]

 ...

 [[32 19 22]
  [32 19 22]
  [32 19 22]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[32 19 22]
  [32 19 22]
  [32 19 22]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[32 19 22]
  [32 19 22]
  [3

frame [[[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]]
frame [[[30 18 17]
  [30 18 17]
  [31 19 18]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[30 18 17]
  [30 18 17]
  [31 19 18]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[30 18 17]
  [30 18 17]
  [31 19 18]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[30 18 17]
  [30 18 17]
  [3

frame [[[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [36 33  0]
  [36 38  0]
  [36 38  0]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [36 38  0]
  [36 44  0]
  [36 44  0]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [36 38  0]
  [36 44  0]
  [36 44  0]]]
frame [[[32 19 22]
  [32 19 22]
  [32 19 22]
  ...
  [34 22 21]
  [34 22 21]
  [34 22 21]]

 [[32 19 22]
  [32 19 22]
  [32 19 22]
  ...
  [34 22 21]
  [34 22 21]
  [34 22 21]]

 [[32 19 22]
  [32 19 22]
  [32 19 22]
  ...
  [34 22 21]
  [34 22 21]
  [34 22 21]]

 ...

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[30 18 17]
  [30 18 17]
  [3

frame [[[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 33  0]
  [36 38  0]
  [36 38  0]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 38  0]
  [36 44  0]
  [36 44  0]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 38  0]
  [36 44  0]
  [36 44  0]]]
frame [[[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[30 18 17]
  [30 18 17]
  [3

frame [[[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 20 19]
  [36 20 19]
  [36 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 20 19]
  [36 20 19]
  [36 20 19]]

 [[32 20 19]
  [32 20 19]
  [32 20 19]
  ...
  [36 20 19]
  [36 20 19]
  [36 20 19]]

 ...

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [30 30  0]
  [30 34  0]
  [30 34  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [30 34  0]
  [30 41  0]
  [30 41  0]]

 [[30 18 17]
  [30 18 17]
  [30 18 17]
  ...
  [30 34  0]
  [30 41  0]
  [30 41  0]]]
frame [[[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 [[36 20 19]
  [36 20 19]
  [36 20 19]
  ...
  [32 20 19]
  [32 20 19]
  [32 20 19]]

 ...

 [[38 22 21]
  [38 22 21]
  [38 22 21]
  ...
  [32 33  0]
  [32 38  0]
  [32 38  0]]

 [[38 22 21]
  [38 22 21]
  [38 22 21]
  ...
  [32 38  0]
  [32 44  0]
  [32 44  0]]

 [[38 22 21]
  [38 22 21]
  [3